In [ ]:
"""
┌──────────────────────────────┐          ┌──────────────────────────────┐
│          Usuario             │          │    Respuesta generada en     │
│ Sube un PDF y hace preguntas │          │      lenguaje natural        │
└────────────┬─────────────────┘          └────────────▲─────────────────┘
             │                                         │
             ▼                                         │
┌──────────────────────────────┐          ┌──────────────────────────────┐
│          Streamlit           │          │  LangChain + Ollama (gemma)  │
│ Interfaz web (UI/UX)         │          │  LLM local via Ollama        │
└────────────┬────────────────┘           └────────────▲─────────────────┘
             │                                         │
             ▼                                         │
┌──────────────────────────────┐          ┌──────────────────────────────┐
│     Extracción de texto      │          │      Búsqueda y envío        │
│    desde PDF (pdfplumber)    │          │     del contexto al LLM      │
└────────────┬────────────────┘           └────────────▲─────────────────┘
             │                                         │
             ▼                                         │
┌──────────────────────────────┐          ┌──────────────────────────────┐
│  Generación de embeddings    │          │        Indexación semántica  │
│ con HuggingFaceEmbeddings    │ ────►    │          usando FAISS        │
└──────────────────────────────┘          └──────────────────────────────┘

"""

In [2]:
from langdetect import detect, DetectorFactory
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationSummaryMemory
from langchain.prompts import PromptTemplate
from PyPDF2 import PdfReader

from langchain_community.llms import Ollama
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader

In [3]:
# Step 1: Load & extract text
pdf_path = "..\\data\\La ciudad de Nuvora.pdf"
reader = PdfReader(pdf_path)
raw_text = "\n".join(page.extract_text() for page in reader.pages if page.extract_text())

In [6]:
try:
    detected_lang = detect(raw_text[:1000])
except:
    detected_lang = "unknown"
language = detected_lang
language

'es'

In [7]:
OLLAMA_MODEL = "gemma:2b"
CHUNK_SIZE = 1000
CHUNK_OVERLAP = 100
NUM_TOP_DOCS = 3

# Load local LLM
def load_llm():
    return Ollama(model=OLLAMA_MODEL, temperature=0.1)

# Load PDF, split and embed
def process_pdf(path):
    loader = PyPDFLoader(path)
    pages = loader.load()
    splitter = RecursiveCharacterTextSplitter(chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP)
    docs = splitter.split_documents(pages)

    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    vectorstore = FAISS.from_documents(docs, embeddings)
    retriever = vectorstore.as_retriever()
    return retriever

In [9]:
# Load LLM and retriever
llm = load_llm()
retriever = process_pdf(pdf_path)

In [11]:
# Get summary
summary = llm.predict(f"Resuma el documento en 5 puntos:\n{raw_text[:3000]}")
summary

'1. La ciudad de Nuvora se convirtió en el primer asentamiento humano construido completamente sobre una plataforma aérea sustentada por energía cuántica.\n\n\n2. La ciudad no tiene gobierno central, las decisiones se toman mediante votación ciudadana a través del Círculo de Conciencia Colectiva.\n\n\n3. La economía de Nuvora es postmonetaria, en lugar de dinero, los ciudadanos intercambian "tiempo de dedicación".\n\n\n4. En Nuvora no existen cárceles, en su lugar, quienes cometen delitos deben participar en programas de reintegración sensorial.\n\n\n5. El éxito de Nuvora ha despertado debates en otras regiones del planeta, donde se discute si replicar su modelo podría resolver los problemas de las sociedades contemporáneas o si su aparente perfección encubre una forma más sofisticada de control social.'

In [ ]:
CUSTOM_PROMPT = """
Eres un asistente inteligente con acceso a un documento y conocimientos generales.
Intente siempre responder utilizando el documento, pero si éste no contiene la respuesta,
no dude en responder con conocimientos generales útiles.

Mantén un tono amistoso y conversacional, y que las respuestas sean breves y claras.

{context}

Chat History:
{chat_history}

User: {question}
"""

In [20]:
# Setup memory and prompt
chat_history = []
memory = ConversationSummaryMemory(llm=llm, memory_key="chat_history", return_messages=True)
prompt = PromptTemplate(
    input_variables=["chat_history", "question", "context"],
    template=CUSTOM_PROMPT
    )

In [21]:
# Build Conversational QA chain
qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=memory,
    combine_docs_chain_kwargs={"prompt": prompt},
    verbose=False
    )

In [ ]:
user_input = "¿Quiénes diseñaron la ciudad de Nuvora?"
response = qa_chain.invoke(user_input)
print(response)

In [ ]:
""" ¿En qué año fue construida la ciudad de Nuvora?
    ¿Quiénes diseñaron la ciudad de Nuvora?
    ¿Qué usan los ciudadanos de Nuvora en lugar de dinero?
    ¿Qué hacen con las personas que cometen delitos en Nuvora?
"""